In [ ]:
import qckt
from qckt.backend import *
import qckt.noisemodel as ns
import numpy as np

# Getting started

## create a simple bell-state circuit

In [ ]:
cnotckt = qckt.QCkt(2, 2)

cnotckt.H(0)
cnotckt.CX(0,1)
cnotckt.M([0,1],[0,1])

cnotckt.draw()

## use NoiseModel mechanism to overlay a noise model on the circuit; adding noise after each gate

In [ ]:
noise_model = ns.NoiseModel(kraus_opseq_allgates=ns.bit_flip(0.1))
cnotckt.add_noise_model(noise_model=noise_model)
cnotckt.draw()

## running the circuit

### lets see what all backend engines we have available
the backend engines nisqsim-eng and nisqsim-deb are density matrix based simulators, and support noise simulation. The engines qsim-eng and qsim-deb are state vector based and do not support noise simulation.

In [ ]:
qsimSvc().listInstances()

### create a job, and run it on backend that supports noise simulation
go ahead and also try running it on a backend that does not support noise simulation (the noise aspects will be ignored)

In [ ]:
job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)

### get and present the results

In [ ]:
print("READ OUT STATE VECTOR: ")
print(job.get_svec())
print("READ OUT CREGISTER: ", end="")
print( job.get_creg()[0])
job.plot_counts()

# More on NoiseModel

## `NoiseModel` class
`NoiseModel` has three fields
* `kraus_opseq_init` - can be `KrausOperator` or `KrausOperatorSequence` or `None` (default)
* `kraus_opseq_allgates` - can be `KrausOperator` or `KrausOperatorSequence` or `None` (default)
* `kraus_opseq_qubits` - can be `KrausOperatorApplierSequense` or `None` (default)
* `kraus_opseq_allsteps` - can be `KrausOperatorApplierSequense` or `None` (default)

First, about `KrausOperator`, `KrausOperatorSequence`, `KrausOperatorApplierSequence`
* `KrausOperator` - is a representation of a kraus operator. Includes three fields
    * a sequence of $(k_i,p_i)$, to enable computing of $\sum_i(p_i k_i \rho k_i^\dagger)$
    * `name` for that operator (e.g., 'PF(0.10)' for a kraus operator bit-flip with $p_i = 0.1$)
    * `nqubits`, the number of qubits this kraus operator acts on.
* `KrausOperatorSequence` - often a sequence of `KrauseOperator`s are applied together. For that reason `KrausOperatorSequence` is used. It simply stores a sequence of `KrausOperator`s. It also has a `name` field to represent the list of `KrausOperator`s (e.g., 'PF(0.10),BF(0.05)')
* `KrausOperatorApplierSequence` - specification of applying a kraus operator onto certain qubits is refered to as `applier`. E.g., `(BF(0.05), [0,1,2])` specifies applyting bit-flip with probability of 0.05 to qubits 0, 1 and 2. Again, in general there can be a need to have a sequence of `appliers`, hence the `KrausOperatorApplierSequence`. As  common thread, this also has a `name` field for the entire applier sequence.

Back to `NoiseModel`.
* `kraus_opseq_init` specifies a `KrausOperator`, or a `KrauseOperatorSequence` to be applied to *all* qubits at the initialization of the quantum computer. The kraus operators specified in this *must* be 1-qubit operators.
* `kraus_opseq_allgates` specifies a `KrausOperator`, or a `KrauseOperatorSequence` to be applied immediately after *each* gate in the circuit. The kraus operators specified in this *must* be 1-qubit operators.
* `kraus_opseq_qubits` specifies a `KrausOperatorApplierSequence` to be applied immediately after *each* gate in the circuit. The qubits to which the operator is actually applied is the intersection of the set of qubits that gate is applied to and the qubits in individual applier entries. The kraus operators specified in this *must* be 1-qubit operators.
* `kraus_opseq_allsteps` specifies a `KrausOperatorApplierSequence` to be applied immediately after *each* gate in the circuit. The qubits to which the operator is actually applied is given by the *applier*. The kraus operators specified in this must be either 1-qubit operators, or if multi-qubit operator whose size matches the number of qubits.

## 1-qubit, 2-qubit (and potentially, multi-qubit) kraus operators
kraus operators can be 1-qubit, 2-qubit, or in general, multi-qubit operators. At the time of application of the operator the size of the operator is checked -
* if the operator is 1-qubit sized, it gets *broadcast* to all the qubits it is being applied to. That is, it is individually applied to each of the qubits in the applier.
* if not 1-qubit, then the number of qubits *must* match the size of the operator, else an exception is thrown.

# Applying a more general NoiseModel to the circuit

## apply init, qubits, and allgates components of NoiseModel
note that between `kraus_opseq_qubits` and `kraus_opseq_allgates`, the *qubits* noise is applied first and then *allgates* noise is applied.

In [ ]:
# start with the same simple circuit
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.CX(0,1)
cnotckt.M([0,1],[0,1])
cnotckt.draw()

noise_model = ns.NoiseModel(
    kraus_opseq_init=ns.generalized_amplitude_damping(probability=0.1,gamma=0.05),
    kraus_opseq_allgates=ns.bit_flip(0.1),
    kraus_opseq_qubits=ns.KrausOperatorApplierSequense(ns.phase_damping(gamma=0.1),[0]),
    )
cnotckt.add_noise_model(noise_model=noise_model)
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

## apply allsteps component of NoiseModel

In [ ]:
# start with the same simple circuit
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.CX(0,1)
cnotckt.M([0,1],[0,1])
cnotckt.draw()

noise_model = ns.NoiseModel(
    kraus_opseq_allsteps=ns.KrausOperatorApplierSequense(
        noise_ops=ns.KrausOperatorSequence(
            ns.bit_flip(0.1),
            ns.phase_flip(0.1),
            ),
        qubit_list=[1]
        )
    )
cnotckt.add_noise_model(noise_model=noise_model)
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

## apply noise to all the gates of a type
In  addition to `NoiseModel`, another way to apply noise globally is to specify noise for individual gate types using `.add_noise_to_all()`.

**Note: Global Noise speccification applies to the entire circuit**. As you will notice, the global noise specifications, `NoiseModel` as well as `.add_noise_to_all()`, are applied to the entire circuit irrespective of whether they are specified -- at the beginning, middle, or end of the circuit definition.

**Note: 2-qubit and multi-qubit kraus operators can be specified here**. If number of qubits match, the kraus operator(s) given in `noise_op` can be 2-qubit or mult-quibit kraus operators.

In [ ]:
# same simple circuit, added another H(1) at the end just to illustrate the `.add_noise_to_all()` functionality
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.CX(0,1)
cnotckt.H(1)
cnotckt.M([0,1],[0,1])

# apply noise to all H gates
cnotckt.H.add_noise_to_all(ns.bit_flip(0.15))
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

## nothing different with custom gates

In [ ]:
# same simple circuit
cnotckt = qckt.QCkt(2, 2)
sqr2 = np.sqrt(2)
cnotckt.custom_gate('myH', np.matrix([[1/sqr2,1/sqr2],[1/sqr2,-1/sqr2]],dtype=complex))
cnotckt.myH(0)
cnotckt.CX(0,1)
cnotckt.myH(1)
cnotckt.M([0,1],[0,1])

# apply noise to all H gates
cnotckt.myH.add_noise_to_all(ns.bit_flip(0.15))
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

# Adding noise to circuit diretly as opposed to overlaying global noise model on a circuit
While overlaying global noise model (NoiseModel, and circuit.GATE.add_noise_to_all()) is expected to be more convinient, adding noise while building the circuit is also supported.
* `circuit.NOISE(noise_op, qubits_list)` - this adds noise at a step in the circuit. `noise_op` can be `KrausOperator` or `KrausOperatorSequence`, and `qubits_list` is a list of qubits on which to apply this noise.
* `circuit.GATE(qubits).add_noise(noise_op)` - this adds noise to the specific `GATE` instance. Of course, `GATE` would be any built-in or custom gate. `noise_op` can be `KrausOperator` or `KrausOperatorSequence`, and that noise is applied to the qubits of this gate instance.

**Note: 2-qubit and multi-qubit kraus operators can be specified here**. If number of qubits match, the kraus operator(s) given in `noise_op` can be 2-qubit or mult-quibit kraus operators in each of the above.

As you can see, the noise is *hardcoded* in the circuit using these, however, might be useful in specific situations.

In [ ]:
# same simple circuit - with noise directly added; added to give the same effect as in the circuit above
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.NOISE(ns.bit_flip(0.15),[0])
cnotckt.CX(0,1)
cnotckt.H(1).add_noise(ns.bit_flip(0.15))
cnotckt.M([0,1],[0,1])
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

# 2-qubit kraus operators


## adding 2-qubit noise to using `.add_noise_to_all()`

In [ ]:
# start with the same simple circuit
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.CX(0,1)
cnotckt.M([0,1],[0,1])
cnotckt.draw()

cnotckt.CX.add_noise_to_all(ns.two_qubit_dephasing(probability=0.1))
cnotckt.draw()

job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()

## adding 2-qubit noise directly to a gate instance

In [ ]:
# start with the same simple circuit
cnotckt = qckt.QCkt(2, 2)
cnotckt.H(0)
cnotckt.CX(0,1).add_noise(ns.two_qubit_dephasing(probability=0.1))
cnotckt.M([0,1],[0,1])
cnotckt.draw()


job = qckt.Job(cnotckt, shots=1000)
bk = qsimSvc().getInstance('nisqsim-eng')
bk.runjob(job)
job.plot_counts()